## CONVAE 

In [1]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, KFold
from sklearn import linear_model, metrics, preprocessing
from math import sqrt, floor
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, IsolationForest
from sklearn.metrics import r2_score, make_scorer, accuracy_score, mean_squared_error, mean_absolute_error
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
import tensorflow as tf
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
import time
from sklearn.pipeline import make_pipeline, Pipeline
import math
import keras
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D

import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers, Sequential
from keras.layers import Attention, GlobalMaxPooling1D
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM1D, Conv1D, BatchNormalization, RepeatVector, TimeDistributed, AveragePooling1D, UpSampling1D
import random
import pickle
import statistics
from tcn import TCN
from sklearn.metrics import f1_score

2023-10-16 13:19:40.383932: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-16 13:19:40.383976: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
import ast
def temporalize(X, lookback=8):

    output_X = []
    #output_y = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            # Gather the past records upto the lookback period
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
        #output_y.append(y[i + lookback + 1])
    #return np.squeeze(np.array(output_X)), np.array(output_y)
    return np.squeeze(np.array(output_X))
def getHyperParams(file, df_hyp):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['Inverter'] == file]['Feature Importance'].values[0])
    return param_dict['model__batch_size'], param_dict['model__encoding_dim'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2'] 

In [5]:
PID_RESULTSPATH_TOTAL_TRAIN = '/mnt/stud/home/vpulagura/experiment/PID2Weeks/PID_2W_NewResults/Results/Iter1/train/'
PID_RESULTSPATH_TOTAL_TEST = '/mnt/stud/home/vpulagura/experiment/PID2Weeks/PID_2W_NewResults/Results/Iter1/test/'
# PID_RESULTSPATH_TOTAL_TRAIN = '/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Results/Iter1/train/'
# PID_RESULTSPATH_TOTAL_TEST = '/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Results/Iter1/test/'
source_list = [12, 4, 6, 14, 10, 18, 5, 8, 3, 0, 11, 2]
total_list = [0, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 17, 18]



In [10]:
df_hyp = pd.read_csv('/mnt/stud/home/vpulagura/experiment/PID2Weeks/PID_2W_NewResults/ConvAE_HYP.csv')

callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
# source_dict_results_test =  {}
# source_dict_results_train = {}
# for file in total_list:
#     inv = 'inv_'+str(file)
#     xtrain, xtest, ytrain, ytest = get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)
#     xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
#     xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])
#     model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(file, df_hyp)                                               
#     model  = ConvAEModel(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
#     model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=2,validation_split=0.1, callbacks=callback1)
#     test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, model)
#     lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
#     source_dict_results_test[file] = test_df
#     source_dict_results_train[file] = train_df
# with open(PID_RESULTSPATH_TOTAL_TEST+'test_STL_ConvAE.pkl', 'wb') as f:
#     pickle.dump(source_dict_results_test, f)
# with open(PID_RESULTSPATH_TOTAL_TRAIN+'train_STL_ConvAE.pkl', 'wb') as f:
#     pickle.dump(source_dict_results_train, f)
train_merge_df = data_unimodel(source_list, xtrain_dfs)
random.shuffle(train_merge_df)
train_merge_df = train_merge_df.reshape(train_merge_df.shape[0], 1, train_merge_df.shape[1])
model = ConvAEModel(layer1=128, layer2=32, encoding_dim=32)
model.fit(train_merge_df, train_merge_df, epochs=200, batch_size=128, verbose=2,validation_split=0.1, callbacks=callback1)
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
mtl_dict_results_test =  {}
mtl_dict_results_train = {}
for file in total_list:
    inv = 'inv_'+str(file)
    xtrain, xtest, ytrain, ytest = get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)
    xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
    xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])
    test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, model)
    lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
    mtl_dict_results_test[file] = test_df
    mtl_dict_results_train[file] = train_df
with open(PID_RESULTSPATH_TOTAL_TEST+'test_MTL_ConvAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_test, f)
with open(PID_RESULTSPATH_TOTAL_TRAIN+'train_MTL_ConvAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_train, f)

Epoch 1/200
230/230 - 2s - loss: 0.0530 - val_loss: 0.0239 - 2s/epoch - 8ms/step
Epoch 2/200
230/230 - 1s - loss: 0.0184 - val_loss: 0.0189 - 996ms/epoch - 4ms/step
Epoch 3/200
230/230 - 1s - loss: 0.0135 - val_loss: 0.0131 - 1s/epoch - 4ms/step
Epoch 4/200
230/230 - 1s - loss: 0.0090 - val_loss: 0.0109 - 1s/epoch - 5ms/step
Epoch 5/200
230/230 - 1s - loss: 0.0069 - val_loss: 0.0100 - 994ms/epoch - 4ms/step
Epoch 6/200
230/230 - 1s - loss: 0.0059 - val_loss: 0.0088 - 1s/epoch - 4ms/step
Epoch 7/200
230/230 - 1s - loss: 0.0048 - val_loss: 0.0077 - 974ms/epoch - 4ms/step
Epoch 8/200
230/230 - 1s - loss: 0.0043 - val_loss: 0.0073 - 988ms/epoch - 4ms/step
Epoch 9/200
230/230 - 1s - loss: 0.0040 - val_loss: 0.0070 - 1s/epoch - 4ms/step
Epoch 10/200
230/230 - 1s - loss: 0.0038 - val_loss: 0.0068 - 971ms/epoch - 4ms/step
Epoch 11/200
230/230 - 1s - loss: 0.0037 - val_loss: 0.0065 - 987ms/epoch - 4ms/step
Epoch 12/200
230/230 - 1s - loss: 0.0035 - val_loss: 0.0060 - 1s/epoch - 4ms/step
Epoch 1

Epoch 97/200
230/230 - 1s - loss: 4.0798e-04 - val_loss: 0.0011 - 939ms/epoch - 4ms/step
Epoch 98/200
230/230 - 1s - loss: 4.1599e-04 - val_loss: 0.0011 - 973ms/epoch - 4ms/step
Epoch 99/200
230/230 - 1s - loss: 4.0384e-04 - val_loss: 0.0010 - 998ms/epoch - 4ms/step
Epoch 100/200
230/230 - 1s - loss: 4.1542e-04 - val_loss: 0.0012 - 978ms/epoch - 4ms/step
Epoch 101/200
230/230 - 1s - loss: 4.0109e-04 - val_loss: 0.0011 - 980ms/epoch - 4ms/step
Epoch 102/200
230/230 - 1s - loss: 4.0760e-04 - val_loss: 0.0011 - 940ms/epoch - 4ms/step
Epoch 103/200
230/230 - 1s - loss: 3.9519e-04 - val_loss: 0.0010 - 963ms/epoch - 4ms/step
Epoch 104/200
230/230 - 1s - loss: 3.9276e-04 - val_loss: 0.0010 - 977ms/epoch - 4ms/step
Epoch 105/200
230/230 - 1s - loss: 3.8429e-04 - val_loss: 9.8510e-04 - 983ms/epoch - 4ms/step
Epoch 106/200
230/230 - 1s - loss: 3.8512e-04 - val_loss: 0.0010 - 964ms/epoch - 4ms/step
Epoch 107/200
230/230 - 1s - loss: 3.8544e-04 - val_loss: 0.0011 - 925ms/epoch - 4ms/step
Epoch 108

Epoch 185/200
230/230 - 1s - loss: 2.5454e-04 - val_loss: 8.0909e-04 - 925ms/epoch - 4ms/step
Epoch 186/200
230/230 - 1s - loss: 2.6230e-04 - val_loss: 7.3366e-04 - 914ms/epoch - 4ms/step
Epoch 187/200
230/230 - 1s - loss: 2.5396e-04 - val_loss: 7.4022e-04 - 924ms/epoch - 4ms/step
Epoch 188/200
230/230 - 1s - loss: 2.5883e-04 - val_loss: 7.3152e-04 - 921ms/epoch - 4ms/step
Epoch 189/200
230/230 - 1s - loss: 2.5620e-04 - val_loss: 7.6974e-04 - 923ms/epoch - 4ms/step
Epoch 190/200
230/230 - 1s - loss: 2.5174e-04 - val_loss: 7.6578e-04 - 911ms/epoch - 4ms/step
Epoch 191/200
230/230 - 1s - loss: 2.6092e-04 - val_loss: 7.8629e-04 - 870ms/epoch - 4ms/step
Epoch 192/200
230/230 - 1s - loss: 2.5390e-04 - val_loss: 7.4700e-04 - 969ms/epoch - 4ms/step
Epoch 193/200
230/230 - 1s - loss: 2.5121e-04 - val_loss: 8.4312e-04 - 925ms/epoch - 4ms/step
Epoch 194/200
230/230 - 1s - loss: 2.8715e-04 - val_loss: 7.3440e-04 - 888ms/epoch - 4ms/step
Epoch 195/200
230/230 - 1s - loss: 2.4496e-04 - val_loss: 7.

## CONVATTNAE

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.metrics import f1_score, make_scorer, mean_squared_error
import tensorflow as tf
import pandas as pd
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D
from tensorflow.keras.layers import  AveragePooling1D, UpSampling1D
import os
import numpy as np
from tensorflow.keras.layers import ConvLSTM1D, Conv1D, BatchNormalization, RepeatVector, TimeDistributed, Reshape,Attention, GlobalMaxPooling1D
import random
import pickle
import statistics
from tcn import TCN
import ast
def ConvAttnAEModel(timesteps = 1, n_features=137, layer1=100, layer2=60, encoding_dim=4):
    input = Input(shape=(timesteps, n_features))
    x = Conv1D(filters=layer1, activation="relu",kernel_size=3, padding="same")(input)
    x = Conv1D(filters=layer2,  activation="relu", kernel_size=3,padding="same")(x)
    x = Conv1D(filters=encoding_dim,  activation="relu",kernel_size=3, padding="same")(x)
    attention = Attention()([x, x])
    attention = GlobalMaxPooling1D()(attention)
    attention = Reshape((1, encoding_dim))(attention)
    x = Conv1DTranspose(filters=layer2, activation="relu", kernel_size=3,padding="same")(x)
    x = Conv1DTranspose(filters=layer1,  activation="relu",kernel_size=3, padding="same")(x)
    x = Conv1DTranspose(filters=n_features,  activation="sigmoid",kernel_size=3, padding="same")(x)
    autoencoder = Model(input, x)
    autoencoder.compile(optimizer="adam", loss="mse")
#     print(autoencoder.summary())
    return autoencoder
def temporalize(X, lookback=8):

    output_X = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
    return np.squeeze(np.array(output_X))
def getHyperParams(file, df_hyp):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['Inverter'] == file]['Feature Importance'].values[0])
    return param_dict['model__batch_size'], param_dict['model__encoding_dim'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2'] 

def getPIDData():
    path_errors = '/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/errortimestamps.csv'
    path_features = '/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/featureslist.csv'
    df_errors = pd.read_csv(path_errors)
    df_features = pd.read_csv(path_features)
    total_features = list(df_features[(df_features['True/False'] == 'TRUE') | (df_features['True/False'] == 'NotAvailable')]['Features'])
    tot_feat = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/totfeat.csv')
    tot_feat = list(tot_feat['feat'])
    xtrain_dfs = {}
    xtest_dfs = {}
    ytrain_dfs = {}
    ytest_dfs = {}
    for inverter in range(19):
        inv = 'inv_'+str(inverter)
        tot_feat.append('ErrBits')
        df_inv_0 = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/downsampledata_PandasMeanNew/'+str(inverter)+'.csv')
        df_inv_0['Timestamp'] = pd.to_datetime(df_inv_0['Timestamp'])
        df_inv_0.sort_values(by='Timestamp', inplace=True)
        df_inv_0 = df_inv_0.set_index(df_inv_0['Timestamp'])
        df_inv_0.shape


        format = '%Y-%m-%d %H:%M:%S'
        error_date = datetime.strptime(df_errors[(df_errors['Inverter'] == inverter) & (df_errors['had_failure'] == True)]['failure_time'].values[0][0:19], format)
        start_date = error_date-timedelta(days= 44)
        end_date = error_date+timedelta(days= 1*30)
        split_date = error_date-timedelta(days= 1*30)

        df_inv_0 = df_inv_0[(df_inv_0.index > start_date) & (df_inv_0.index < end_date)]
        df_inv_0 = pd.get_dummies(df_inv_0)

        for feat in tot_feat:
            if feat not in df_inv_0.columns:
                df_inv_0[feat] = 0
        df_inv_0 = df_inv_0[tot_feat]
        df_inv_0 = df_inv_0.dropna()
        df_target = df_inv_0[['ErrBits']]
        tot_feat.remove('ErrBits')
        df_inv_0 = df_inv_0[tot_feat]

        xtrain, xtest = df_inv_0[(df_inv_0.index >= start_date) & (df_inv_0.index< split_date)], df_inv_0[(df_inv_0.index >= split_date) & (df_inv_0.index< end_date)]
        ytrain, ytest = df_target[(df_target.index >= start_date) & (df_target.index< split_date)], df_target[(df_target.index >= split_date) & (df_target.index< end_date)]
        xtrain_dfs[inv] = xtrain
        xtest_dfs[inv] = xtest
        ytrain_dfs[inv] = ytrain
        ytest_dfs[inv] = ytest
    return xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs
def get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs):
    xtrain, xtest = xtrain_dfs[inv], xtest_dfs[inv]
    ytrain, ytest = ytrain_dfs[inv], ytest_dfs[inv]
    X_scaler = MinMaxScaler()
    X_scaler.fit(xtrain)
    xtrain = X_scaler.transform(xtrain)
    xtest = X_scaler.transform(xtest)
    X_scalar = MinMaxScaler()
    X_scalar.fit(xtrain)
    xtrain, xtest = X_scalar.transform(xtrain), X_scalar.transform(xtest)
    return xtrain, xtest, ytrain, ytest

def other_evaluation(xtrain, xtest, ytest, ytrain, model):
    test_enc = model.predict(xtest)
    train_enc = model.predict(xtrain)
    test_mses = np.mean(np.power(flatten(xtest)- flatten(test_enc), 2), axis=1)
    train_mses = np.mean(np.power(flatten(xtrain)- flatten(train_enc), 2), axis=1)
    test_maes = np.mean(flatten(xtest)- flatten(test_enc), axis=1)
    train_maes = np.mean(flatten(xtrain)- flatten(train_enc), axis=1)
    mse_threshold = np.max(train_mses)
    mae_threshold = np.max(train_maes)
    ypred_mse = np.where(test_mses>mse_threshold,1,0)
    ypred_mae = np.where(test_maes>mae_threshold,1,0)
    test_df = pd.DataFrame(ytest)
    test_df['mse_scores'] = test_mses
    test_df['ytrue'] = np.where(test_df['ErrBits']>0, 1, 0)
    test_df['mae_scores'] = test_maes
    test_df['ypred_mse'] = ypred_mse
    test_df['ypred_mae'] = ypred_mae
    test_df = test_df[['ytrue','ypred_mse','ypred_mae','mse_scores', 'mae_scores']]
    
    train_df = pd.DataFrame(ytrain)
    train_df['mse_scores'] = train_mses
    train_df['ytrue'] = np.where(train_df['ErrBits']>0, 1, 0)
    train_df['mae_scores'] = train_maes
    train_df = train_df[['ytrue','mse_scores', 'mae_scores']]
    return test_df, train_df
def data_unimodel(source_list, xtrain_dfs):
    train_merge = pd.DataFrame()
    for inverter in source_list:
        X = xtrain_dfs[f'inv_{inverter}']
        X_scaler = MinMaxScaler()
        X_scaler.fit(X)
        xtrain = X_scaler.transform(X)
        df_train = pd.DataFrame(xtrain)
        train_merge = pd.concat([train_merge, df_train], axis=0)
    train_merge_df = np.array(train_merge)
    return train_merge_df
def flatten(X):
    flattened_X = np.empty((X.shape[0], X.shape[2])) 
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)
source_list = [12, 4, 6, 14, 10, 18, 5, 8, 3, 0, 11, 2]
total_list = [0, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 17, 18]

xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs = getPIDData()
for i in range(1,4):
    df_hyp = pd.read_csv('/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/ConvAttnAE_HYP.csv')
    PID_RESULTSPATH_TOTAL_TRAIN = f'/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Results/Iter{i}/train/'
    PID_RESULTSPATH_TOTAL_TEST = f'/mnt/work/digitwin/subset_data_for_experiment/PID_2WExp/Results/Iter{i}/test/'
    callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
    lookback = 8
#     source_dict_results_test =  {}
#     source_dict_results_train = {}
#     for file in total_list:
#         inv = f'inv_{file}'
#         xtrain, xtest, ytrain, ytest = get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)
#         xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
#         xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])
# #         ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
#         model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(file, df_hyp)                                               
#         model  = ConvAttnAEModel(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
#         model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=2,validation_split=0.1, callbacks=callback1)
#         test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, model)
#         source_dict_results_test[file] = test_df
#         source_dict_results_train[file] = train_df
#     with open(PID_RESULTSPATH_TOTAL_TEST+'test_STL_ConvAttnAE.pkl', 'wb') as f:
#         pickle.dump(source_dict_results_test, f)
#     with open(PID_RESULTSPATH_TOTAL_TRAIN+'train_STL_ConvAttnAE.pkl', 'wb') as f:
#         pickle.dump(source_dict_results_train, f)
    train_merge_df = data_unimodel(source_list, xtrain_dfs)
    random.shuffle(train_merge_df)
    train_merge_df = train_merge_df.reshape(train_merge_df.shape[0], 1, train_merge_df.shape[1])

    model = ConvAttnAEModel(layer1=128, layer2=32, encoding_dim=32)
    model.fit(train_merge_df, train_merge_df, epochs=200, batch_size=128, verbose=2,validation_split=0.1, callbacks=callback1)
    lookback = 8
    mtl_dict_results_test =  {}
    mtl_dict_results_train = {}
    for file in total_list:
        inv = 'inv_'+str(file)
        xtrain, xtest, ytrain, ytest = get_train_test_data(inv, xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)
        xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
        xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])
#         ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
        test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, model)
        mtl_dict_results_test[file] = test_df
        mtl_dict_results_train[file] = train_df
    with open(PID_RESULTSPATH_TOTAL_TEST+'test_MTL_ConvAttnAE.pkl', 'wb') as f:
        pickle.dump(mtl_dict_results_test, f)
    with open(PID_RESULTSPATH_TOTAL_TRAIN+'train_MTL_ConvAttnAE.pkl', 'wb') as f:
        pickle.dump(mtl_dict_results_train, f)